
# Module 2 - Salary Function Assignment 




## 1) Import Data



In [1]:

# Imports and basic settings
import os, zipfile, re
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 120)

DATA_FILE = 'Total.csv'
print('Expecting file:', DATA_FILE)


Expecting file: Total.csv


In [2]:

# Load the dataset (basic). More robust error handling is shown later in step 4.
df = pd.read_csv(DATA_FILE)
print(f'Loaded: {len(df):,} rows, {len(df.columns)} columns')
df.head()


Loaded: 312,882 rows, 9 columns


C:\Users\Admin\AppData\Local\Temp\ipykernel_12432\2138645349.py:2: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_FILE)


,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.0,400184.25,Not Provided,567595.43,567595.43,2011
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,Not Provided,538909.28,538909.28,2011
2,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,Not Provided,335279.91,335279.91,2011
3,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.0,56120.71,198306.9,Not Provided,332343.61,332343.61,2011
4,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737.0,182234.59,Not Provided,326373.19,326373.19,2011



## 2) Create Employee Function



In [3]:


def _detect_name_column(df):
    candidates = ['EmployeeName','Name','FullName','Employee','StaffName']
    lower_map = {c.lower(): c for c in df.columns}
    for c in candidates:
        if c.lower() in lower_map:
            return lower_map[c.lower()]
    # fuzzy fallback
    for col in df.columns:
        cl = col.lower()
        if cl.endswith('name') or cl.startswith('name') or ('employee' in cl and 'name' in cl):
            return col
    return None

NAME_COL = _detect_name_column(df)
print('Detected name column:', NAME_COL)

def get_employee_details(name: str) -> pd.DataFrame:
    if not isinstance(name, str) or not name.strip():
        raise ValueError('Please provide a non-empty employee name (string).')
    if NAME_COL is None:
        raise KeyError('Could not detect an employee name column in the dataset.')
    mask = df[NAME_COL].astype(str).str.casefold() == name.casefold()
    result = df.loc[mask]
    if result.empty:
        raise LookupError(f'Employee not found: {name}')
    return result


Detected name column: EmployeeName


In [4]:
get_employee_details("STEPHEN TACCHINI")

,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year
225,STEPHEN TACCHINI,CAPTAIN III (POLICE DEPARTMENT),188341.6,2258.63,12252.44,Not Provided,202852.67,202852.67,2011
36195,Stephen Tacchini,Captain 3,104404.0,0.0,204290.09,24688.54,308694.09,333382.63,2012



## 3) Data Processing with Dictionary


In [ ]:

# Detect useful numeric columns

def _detect_col(df, candidates):
    low = {c.lower(): c for c in df.columns}
    for c in candidates:
        if c.lower() in low:
            return low[c.lower()]
    return None

COL_BASE = _detect_col(df, ['BasePay','BasicPay','Base_Salary','Basic','Salary'])
COL_OT   = _detect_col(df, ['Overtime','OvertimePay','OT','OT_Pay'])
COL_ALLOW= _detect_col(df, ['Allowance','Allowances','Benefits'])
COL_DED  = _detect_col(df, ['Deduction','Deductions','Withholding','Tax'])
COL_HRS  = _detect_col(df, ['HoursWorked','Hours','Hrs'])
COL_RATE = _detect_col(df, ['HourlyRate','Rate','PayRate'])
print('Detected columns ->', COL_BASE, COL_OT, COL_ALLOW, COL_DED, COL_HRS, COL_RATE)

def _to_float(row, col):
    if col and col in row and pd.notna(row[col]):
        try:
            return float(row[col])
        except Exception:
            return 0.0
    return 0.0

def employee_to_dict(name: str) -> dict:
    rec = get_employee_details(name)
    row = rec.iloc[0]
    base = _to_float(row, COL_BASE)
    ot   = _to_float(row, COL_OT)
    allow= _to_float(row, COL_ALLOW)
    ded  = _to_float(row, COL_DED)
    hrs  = _to_float(row, COL_HRS)
    rate = _to_float(row, COL_RATE)

    if base == 0 and hrs > 0 and rate > 0:
        base = hrs * rate

    gross = base + ot + allow
    tax = 0.10 * gross
    pension = 0.05 * gross
    net = gross - tax - pension - ded

    out = {
        'EmployeeName': row[NAME_COL] if NAME_COL else name,
        'base': base,
        'overtime': ot,
        'allowances': allow,
        'other_deductions': ded,
        'gross': gross,
        'tax': tax,
        'pension': pension,
        'net': net
    }
    # Add optional informative fields if present
    for k in ['EmployeeID','EmpID','ID','StaffID','Department','Title','Position']:
        if k in rec.columns:
            out[k] = row[k]
    return out



## 4) Error Handling


In [6]:

# Example error-handling patterns:
try:
    _ = get_employee_details('@@@_unlikely_name_@@@')
except LookupError as e:
    print('Handled LookupError:', e)

try:
    _ = employee_to_dict('@@@_unlikely_name_@@@')
except Exception as e:
    print('Handled error when converting to dict:', e)


Handled LookupError: Employee not found: @@@_unlikely_name_@@@
Handled error when converting to dict: Employee not found: @@@_unlikely_name_@@@



## 5) Export Employee Details.


In [7]:

OUTPUT_DIR = 'Employee Profile'
ZIP_NAME = 'Employee_Profile.zip'

def export_employee_profile(name: str, output_dir: str = OUTPUT_DIR, zip_name: str = ZIP_NAME) -> str:
    # Validate employee exists
    _ = get_employee_details(name)
    data = employee_to_dict(name)
    os.makedirs(output_dir, exist_ok=True)
    safe = re.sub(r'[^A-Za-z0-9_\-]', '_', name.strip())
    csv_path = os.path.join(output_dir, f'{safe}_profile.csv')
    pd.DataFrame([data]).to_csv(csv_path, index=False)
    # Create/overwrite the zip with folder contents
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zf:
        for file in os.listdir(output_dir):
            zf.write(os.path.join(output_dir, file))
    return f'Exported to {csv_path} and zipped as {zip_name}'


In [29]:

# Replace with a real, existing employee name as it appears in your file
export_employee_profile("NATHANIEL FORD")


'Exported to Employee Profile\\NATHANIEL_FORD_profile.csv and zipped as Employee_Profile.zip'

In [1]:

# 1) Unzip into a dedicated folder
unzip("Employee_Profile.zip", exdir = "Employee_Profile_R")

# 2) List everything that was unzipped
files <- list.files("Employee_Profile_R", recursive = TRUE, full.names = TRUE)
print(files)

# 3) Filter to CSVs only (in case there are multiple)
csvs <- files[grepl("\\.csv$", files, ignore.case = TRUE)]
if (length(csvs) == 0) {
  stop("No CSV files found in Employee_Profile_R. Check that the ZIP contains a CSV.")
}

# 4) Pick the one you want (first CSV by default)
csv_path <- csvs[1]
cat("Using CSV:", csv_path, "\n")

# 5) Read it
df <- read.csv(csv_path, stringsAsFactors = FALSE)
print(head(df))


[1] "Employee_Profile_R/Employee Profile/NATHANIEL_FORD_profile.csv"
Using CSV: Employee_Profile_R/Employee Profile/NATHANIEL_FORD_profile.csv 
    EmployeeName     base overtime allowances other_deductions    gross
1 NATHANIEL FORD 167411.2        0          0                0 167411.2
       tax  pension      net
1 16741.12 8370.559 142299.5
